# SPRINT 8. Parte II
> PowerBI + Python

## Nivel 1
Pasar todos los gráficos del S8_1N1 a PowerBI.

El Script de conexión e importación de datos es este:

In [ ]:
import dateutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import configparser
import mysql.connector

cfgFp = open(r'C:\Users\formacio\Downloads\mlg\git_repo\SPRINT8\db.ini', mode='r')
iniFp = configparser.ConfigParser()
iniFp.read_file(cfgFp)
dbCfg = dict(iniFp.items('database'))

# exit()

db = mysql.connector.connect(
    host=dbCfg['host'],
    database=dbCfg['database'],
    user=dbCfg['user'],
    password=dbCfg['password']
)

tr_df   = pd.read_sql("SELECT * FROM `transaction`", db)

user_df = pd.read_sql("SELECT * FROM `user`", db)

co_df = pd.read_sql("SELECT * FROM company", db)

pr_df   = pd.read_sql("SELECT * FROM product", db)
prtr_df = pd.read_sql("SELECT * FROM product_transaction", db)

def age(birthdate):
    dob = pd.to_datetime(birthdate).date() if type(birthdate) is not str \
                  else dateutil.parser.parse(birthdate).date()
    hoy = dateutil.utils.today().date()
    ddiff = hoy - dob
    edad  = int(ddiff.days / 365.25)
    return edad

# Añadimos 'Age' al dataframe user_df, porque se usa en muchos de los gráficos
user_df["age"] = user_df['birthdate'].map(age)


> Para facilitar la localización, organizaré los scripts no por el número de nivel y ejercicio (como está en el doc. **S8.01**), sino por la página en el PowerBI, ya que en ese documento están por "temática", y no estrictamente por Nivel.

### Página 1 - N1: Usuarios y empresas
quí presento tres gráficos, 1 y 2 son la distribución por edad de los usuarios compradores y el otro la distribución regional de las empresas.

#### Gr 1. Distribución de edades de los usuarios.
El script utilizado es:

In [ ]:
import matplotlib.pyplot as plt

dataset.groupby('age')['age'] \
        .count() \
        .plot.bar(
            x='age',
            title="Age distribution",
            ylabel="# of Users",
            xlabel="Age"
        )
plt.show()

#### Gr 2. Histograma de edades
Y el histograma es este. Aún no estoy seguro si dejarlo:

In [ ]:
import matplotlib.pyplot as plt

dataset['age'].plot.hist(
    bins=25,
    color='teal',
    ylabel="# of Users",
    title="N1E1. Distribución de edad de los usuarios"
)

plt.show()

#### Gr 3. Distribución de empresas por países
El código adaptado está a continuación. Notar que como las barras tienen colores según su valor, había que usar una función que devolviese una lista de colores. Esto se repite en otros gráficos, y he tenido que insertar dichas funciones en cada uno de ellos.

In [ ]:
import matplotlib.pyplot as plt

def column_colors(values, color_map: dict):
    cc = []

    for n in values:
        cc.append(get_color(n, color_map))
    return cc

def get_color(number, color_map):
    sorted_keys = sorted(key for key in color_map.keys() if isinstance(key, int))

    if number < sorted_keys[0]:
        return color_map[sorted_keys[0]]
    elif number > sorted_keys[-1]:
        return color_map['else']

    for key in sorted_keys:
        if number < key:
            return color_map[key]

    return color_map[sorted_keys[-1]]

pastel_3colors = {4: '#f99', 8: '#99f', 'else': '#9f9'}
graph_data     = dataset.groupby('country')['country'] \
                        .count()  \
                        .sort_values(ascending=False)

co_by_country = graph_data.plot.bar(
            title="N1E3. Distribución de empresas por país",
            figsize=(14,11),
            x='age',
            rot=60,
            ylabel="Empresas",
            xlabel="País",
            color=column_colors(graph_data, pastel_3colors)
        )
plt.show()

### Página 2 - N1: Transacciones
En esta página se hace un análisis de las transacciones según diversos parámetros.

#### Gr 4. Resultado de las tranascciones
Análisis simple de operaciones declinadas y completadas. Código:

In [ ]:
import matplotlib.pyplot as plt

decl_df = dataset.groupby('declined').count()

decl_df = decl_df.rename({0: "Completed", 1: "Declined"}, axis='index')['transaction_id']
decl_df.plot.bar(
    title="N1E4. Transactiones\ndeclinadas",
    figsize=(3,6),
    xlabel="Status",
    color=['green', 'red'],
    rot=0
)

plt.show()

#### Gr 5. Resultado transacciones por países
Se detalla y analiza la distribución anterior por países. Podría hacerse por empresa, pero sería un listado largo, apropiado para una tabla, o un gráfico del tipo TOPN(). El código es el siguiente:

In [ ]:
import matplotlib.pyplot as plt

cotr_df = dataset
cotr_df['status'] = cotr_df['declined'].map({0: "Completed", 1: "Declined"})

graph_data = cotr_df.groupby(['country', 'status']) \
                    .size() \
                    .unstack(fill_value=0) \
                    .sort_values(by='country', ascending=False)
tr_by_co   = graph_data.plot.barh(
    title='N1E5. Informe de transacciones declinadas por países',
    figsize=(8.4,11),
    xlabel=' ',
    color=['#696', '#966'],
    rot=30
)
plt.show()


#### Gr 6. Distribución de ingresos por país
Se analizan los ingresos, y también lo que no se ha ingresado, según se hayan denegado las operaciones, agrupado por país. Este gráfico es del Nivel 3 de la Parte I, pero está con estos dos porque muestran otro punto de vista de los mismos datos. Código:

In [ ]:
import seaborn as sbn
import matplotlib.pyplot as plt

cotr_df = dataset
cotr_df['declined'] = cotr_df['declined'].map({0: "Completed", 1: "Declined"})
cotr_df['country']  = cotr_df['country'].astype("category")

print(cotr_df['declined'].count())

pl = plt.figure(figsize=(10,9))
vp = sbn.violinplot(
    cotr_df,
    x='amount',
    y="country",
    hue='declined', split=True,
    palette="pastel",
    inner="stick",
    density_norm='count',
    legend="brief",
)

vp = vp.set(
    title="S8.01 N3E1 - Violin plot ventas por país",
    xlabel="Sales",
    ylabel="Country",
)

plt.legend(title="Transaction\ncompletion")
plt.show()

### Página 3 - N1: Ventas
En esta página miramos las ventas, tanto en cantidad como en volumen de ingresos.

#### Gr 7. Ventas e ingresos según edad del comprador
Código:

In [ ]:
from matplotlib import pyplot, ticker
import numpy as np

salesXage = dataset.get(['age', 'amount']) \
                   .groupby('age')['amount'] \
                   .agg(['sum', 'count'])

f, sxa_gr = pyplot.subplots(figsize=(12,8))
ages_list = salesXage.index
bar_width = 0.4
positions = np.arange(len(ages_list))

sxa_gr.bar(
    x=positions,
    height=salesXage['count'],
    width=bar_width,
    label='# of sales',
    color='#5c5'
)

sxa_gr.set_xlabel('Age')
sxa_gr.set_ylabel('# of transactions')
sxa_gr.set_xticks(positions, [str(n) for n in ages_list])

sxa_gr2 = sxa_gr.twinx()
sxa_gr2.bar(
    x=positions+bar_width,
    height=salesXage['sum'] / 1000,
    width=bar_width,
    label='Sales in M€',
    color='#393'
)
sxa_gr2.set_ylabel('Sales in (in thousands of €)')
sxa_gr2.yaxis.set_major_formatter(ticker.FormatStrFormatter('%dM €'))

sxa_gr.set_title("N1E2. Transacciones por edad del usuario")
sxa_gr.legend(loc='upper left')
sxa_gr2.legend(loc='upper right')

pyplot.show()

#### Gr 8. Ventas mensuales
Análisis simple de distribución temporal de las ventas globales. Código:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def column_colors(values, color_map: dict):
    cc = []

    for n in values:
        cc.append(get_color(n, color_map))
    return cc

def get_color(number, color_map):
    sorted_keys = sorted(key for key in color_map.keys() if isinstance(key, int))

    # Primero los valores extremos...
    if number < sorted_keys[0]:
        return color_map[sorted_keys[0]]
    elif number > sorted_keys[-1]:
        return color_map['else']

    # Ahora los valores de las keys
    for key in sorted_keys:
        if number < key:
            return color_map[key]

    return color_map[sorted_keys[-1]]

dataset['transaction_ts'] = pd.to_datetime(dataset['transaction_ts'])

tr_df_agg  = dataset[dataset['declined']==0]  \
                        .loc[:, ['amount', "transaction_ts"]]  \
                        .groupby(pd.Grouper(
                            key='transaction_ts',
                            axis=0,
                            origin='start',
                            sort=True,
                            freq='ME'
                        )) \
                        .agg(['sum', 'count'])
tr_agg     = tr_df_agg['amount']
bar_values = tr_agg['count'].to_list()

semaphor_colors = {
    20:     '#f00',
    40:     '#fb0',
    'else': '#0f0'
}

xticks = [str(d)[:7] for d in tr_agg.index]
cc     = column_colors(bar_values, semaphor_colors)
tr_agg.index = pd.Index(xticks)

graph  = tr_agg['count'].plot.bar(
    figsize=(10,6),
    color=cc,
    rot=60,
    title="N1E4. Ventas mensuales globales",
    ylabel="# of transactions",
    xlabel="Month",
    label="Transactions"
)
graph.spines['top'].set_visible(False)
graph.spines['right'].set_visible(False)
graph.yaxis.set_ticks_position('left')
graph.xaxis.set_ticks_position('bottom')
plt.show()

#### Gr 9. Ventas por países
Similar al anterior, pero por países en lugar de por meses. Código:

In [ ]:
import matplotlib.pyplot as plt
from   matplotlib import ticker

def column_colors(values, color_map: dict):
    cc = []

    for n in values:
        cc.append(get_color(n, color_map))
    return cc

def get_color(number, color_map):
    sorted_keys = sorted(key for key in color_map.keys() if isinstance(key, int))

    if number < sorted_keys[0]:
        return color_map[sorted_keys[0]]
    elif number > sorted_keys[-1]:
        return color_map['else']

    for key in sorted_keys:
        if number < key:
            return color_map[key]

    return color_map[sorted_keys[-1]]

tr_agg  = dataset[dataset['declined']==0]  \
            .loc[:,['amount', 'country']]  \
            .groupby('country')  \
            .agg(['sum', 'count'])

fig, ax = plt.subplots()
xLabels = tr_agg['amount'].index

semaphor_colors = {
    20:     '#f00',
    60:     '#fb0',
    'else': '#0f0'
}

cc    = column_colors(tr_agg['amount']['count'], semaphor_colors)
graph = tr_agg['amount']['count'].plot(
    kind="bar",
    figsize=(17,8.8),
    color=cc,
    rot=60,
    title="N1E6. Transacciones y ventas por países",
    ylabel="# of transactions",
    label="Transactions",
    legend=True,
    ax=ax
)

graph2 = tr_agg['amount']['sum'].plot(
    kind='line',
    secondary_y=True,
    legend=True,
    ylabel='Sales in €',
    color='green',
    marker='o',
    rot=60,
    label="Sales",
    ax=graph
)
graph2.yaxis.set_major_formatter(ticker.FormatStrFormatter('%d €'))

plt.show()

### Página 4: Variables numéricas
Este es el gráfico "raro".

#### Gr 10. Valores numéricos del conjunto de datos
Código:

In [ ]:
import matplotlib.pyplot as plt
from   matplotlib import ticker

cols      = ["product_name", "amount", "weight", "price", "age"]
plotData2 = dataset[['amount', 'weight', 'price', 'age']] \
                .groupby("age").mean()
fig, ax   = plt.subplots()
graph     = plotData2[['amount', 'price']].plot(
    kind="bar",
    ax=ax,
    ylabel="Product Price and Sales",
    legend=True,
)

plotData2.reset_index(inplace=True)

graph2 = plotData2[['weight']].plot(
    kind="line",
    ax=graph,
    legend=True,
    figsize=(12,9),
    secondary_y=True,
    title="N2E1 - Correlación de las variables numéricas",
    ylabel="Weight",
    xlabel="Consumer age",
    color=["red","green"],
    marker="o",
    label="Weight"
)

graph.yaxis.set_major_formatter(ticker.FormatStrFormatter('%d €'))

plt.show()


### Página 5. Ventas
Otras visualizaciones de las ventas por productos, edad, etc.

#### Gr 11. Ventas por edad
En este gráfico vemos un mapa de calor de las ventas según la edad. Código:

In [ ]:
import seaborn           as sbn
import matplotlib.pyplot as plt

salesXage    = dataset.get(['age', 'amount']) \
                      .groupby('age')['amount'] \
                      .agg(['sum', 'count'])

sbnSalesXage = salesXage.rename(columns={
    'count': 'Sales',
    'sum': 'Revenue',
    'age': 'Age'
})

e = sbn.jointplot(
    data=sbnSalesXage,
    y='age',
    x='Sales',
    kind='hist'
)

plt.show()

#### Gr 12. Ventras e ingresos por edad
Este es un PairPlot que relaciona los tres parámetros. Código:

In [ ]:
import seaborn           as sbn
import matplotlib.pyplot as plt

salesXage    = dataset.get(['age', 'amount']) \
                      .groupby('age')['amount'] \
                      .agg(['sum', 'count'])

sbnSalesXage = salesXage.rename(columns={
    'count': 'Sales',
    'sum': 'Revenue',
})
sbnSalesXage['Age'] = salesXage.index
sbn.pairplot(sbnSalesXage)

plt.show()

### Página 6: FacetGrid
Este gráfico relaciona los ingresos por meses, semanas e incluso horas del día.
Código:

In [ ]:
import seaborn as sbn
import pandas  as pd
dataset['transaction_ts'] = pd.to_datetime(dataset['transaction_ts'])

cotr_df = dataset
fg_data = dataset.groupby([
    dataset['transaction_ts'].dt.month,
    dataset['transaction_ts'].dt.day_of_week,
    dataset['transaction_ts'].dt.hour]) \
    .amount.sum()

# No he visto una forma de "desindexar" o "aplanar" la serie,
# así que he tirado de bucles.
mil = fg_data.index.levels
fg_df_raw = []
for mes in mil[0]:
    for dow in mil[1]:
        for hora in mil[2]:
            fg_df_raw.append([
                mes,
                dow,
                hora,
                fg_data[(mes, dow,hora)]
                    if (mes, dow, hora) in fg_data.keys()
                    else 0
            ])

fg_df = pd.DataFrame(
    fg_df_raw,
    columns=['Month', 'Day of Week', 'Hour', 'Revenue']
)

fgd = sbn.FacetGrid(
    fg_df,
    col='Day of Week',
    row='Month',
    hue='Hour'
)

fgd.map_dataframe(sbn.barplot, y='Revenue', x='Hour')

plt.show()